# NumPy

## Memory Management and Performance Fundamentals

In this notebook, we'll explore:

- **Memory Management**: Understanding how NumPy handles memory allocation and why it matters for performance
- **Array Operations**: Efficient vs inefficient operations and their impact on computational speed
- **Storage Strategies**: Comparing different approaches to data persistence and I/O optimization
- **System Resource Monitoring**: Tools and techniques for tracking memory usage in data-intensive applications

### Why This Matters for High Performance Data Analysis

In high performance computing and data analysis, understanding memory patterns is crucial because:

1. **Memory is often the bottleneck** - not CPU speed
2. **Memory allocation/deallocation overhead** can dominate computation time
3. **Cache efficiency** depends on data layout and access patterns
4. **Scalability** requires understanding memory consumption patterns

In [1]:
import psutil
import sys
import os
import numpy as np

### Check Available Memory

Let's start by checking how much RAM is currently available on this system. This is crucial for planning data operations:

In [2]:
print(f"Available RAM: {psutil.virtual_memory().available / (1024 ** 3):.2f} GB")

Available RAM: 2.69 GB


## Large Array Creation and Memory Patterns

### Understanding NumPy Memory Efficiency

NumPy arrays features:

- **Store data in contiguous memory blocks** - enabling efficient CPU cache usage
- **Use homogeneous data types** - eliminating Python object overhead
- **Enable vectorized operations** - leveraging optimized C/Fortran libraries (BLAS, LAPACK)

Let's create a large array to explore memory usage patterns. We'll use `np.arange()` which creates consecutive integers efficiently:

In [3]:
big_array = np.arange(20_000_000)

### Memory Usage Analysis

Understanding how much memory our arrays consume is critical for:

- **Capacity planning** - ensuring we don't exceed available RAM
- **Performance optimization** - predicting cache behavior
- **Scaling decisions** - estimating resource needs for larger datasets

**Note:** 20 million integers × 8 bytes per int64 = ~152 MB theoretical minimum. The actual memory usage may be slightly higher due to Python object overhead.

In [4]:
print(f"Memory used by big_array: {sys.getsizeof(big_array) / (1024 ** 2):.3f} MB")

Memory used by big_array: 152.588 MB


## Array Growth Patterns and Performance Anti-Patterns

### The np.append() Performance Trap

One of the most common performance mistakes in NumPy is repeatedly using `np.append()`. Let's explore why this is problematic:

**Key Issue:** NumPy arrays have **fixed size** - they cannot grow in-place like Python lists. Every `np.append()` operation:

1. **Allocates a completely new array** (size n+1)
2. **Copies all existing data** to the new memory location  
3. **Adds the new element**
4. **Deallocates the old array**

This creates **bottleneck** for building arrays element by element!

Let's observe this behavior by monitoring memory usage as we append elements:

In [5]:
# add one integer at the end of the list, check its size and memory usage
big_array = np.append(big_array, 1)
print(f"Memory used by big_array after appending one element: {sys.getsizeof(big_array) / (1024 ** 2):.3f} MB")

Memory used by big_array after appending one element: 152.588 MB


### Single Append Operation

Watch what happens when we add just **one** element to our large array:

In [6]:
# add one integer at the end of the list, check its size and memory usage
big_array = np.append(big_array, 1)
print(f"Memory used by big_array after appending one element: {sys.getsizeof(big_array) / (1024 ** 2):.3f} MB")

Memory used by big_array after appending one element: 152.588 MB


### Repeated Append Operations

Notice how each append operation requires copying the **entire array**. Let's repeat this and see the cumulative effect:

### Performance Impact of Multiple Appends

Now let's append 100 elements and observe the **catastrophic** memory and performance impact:

**What's happening:** Each append operation:
- Creates a new array with 20,000,000+ elements
- Copies ~152 MB of data 
- Does this 100 times = ~15.2 **GB** of unnecessary memory operations!

In [7]:
# try appending 100 times
for i in range(100):
    size_of_big_array = sys.getsizeof(big_array)
    big_array = np.append(big_array, 1)

In [8]:
print(f"Memory used by big_array after appending one element: {sys.getsizeof(big_array) / (1024 ** 2):.3f} MB")

Memory used by big_array after appending one element: 152.589 MB


## Data Persistence and I/O Optimization

### Efficient Data Storage Strategies

In high performance data analysis, I/O operations often become bottlenecks. NumPy provides several formats for storing arrays efficiently:

- **`.npz` format**: NumPy's native binary format
- **Uncompressed**: Fast write/read, larger file size  
- **Compressed**: Slower write/read, smaller file size

The choice depends on your workflow:
- **Temporary data**: Use uncompressed for speed
- **Long-term storage**: Use compression to save disk space
- **Network transfer**: Compression reduces bandwidth usage

Let's compare both approaches:

### Uncompressed Storage with np.savez()

In [9]:
output_file = 'big_array.npz'
np.savez(output_file, big_array=big_array)

# output file size on disk in MB, memory footprint of numpy array in MB
print(f"Output file size on disk: {os.path.getsize(output_file) / (1024 ** 2):.3f} MB")
print(f"Memory used by big_array: {sys.getsizeof(big_array) / (1024 ** 2):.3f} MB")

Output file size on disk: 152.589 MB
Memory used by big_array: 152.589 MB


### Compressed Storage with np.savez_compressed()

Now let's try the same data with compression enabled:

**Key Insight:** Our array contains sequential integers (0, 1, 2, 3...), which compress **extremely well** because of the predictable pattern. Real-world data may not compress as efficiently.

In [10]:
output_file_compressed = 'big_array_compressed.npz'
np.savez_compressed(output_file_compressed, big_array=big_array)
# output file size on disk in MB, memory footprint of numpy array in MB
print(f"Output file size on disk (compressed): {os.path.getsize(output_file_compressed) / (1024 ** 2):.3f} MB")
print(f"Memory used by big_array: {sys.getsizeof(big_array) / (1024 ** 2):.3f} MB")

Output file size on disk (compressed): 28.873 MB
Memory used by big_array: 152.589 MB


## 5. Key Takeaways and Exercises

### Summary: Critical Performance Lessons

1. **Memory Management**: Always monitor your memory usage in data-intensive applications
2. **Array Growth**: Never use `np.append()` in loops - pre-allocate arrays when possible
3. **I/O Strategy**: Choose appropriate compression based on your use case
4. **Scalability**: Understanding these patterns is essential for working with larger datasets

### 🔍 **Analysis Question**
**Why is repeatedly calling `np.append(big_array, 1)` so inefficient?**

*Think about: memory allocation, data copying, and time complexity*

### 🏋️ **Performance Exercise** 
**Challenge:** Increase the size of the array until it reaches 10 GB of memory usage. 

Requirements:
- Print memory usage each time the array grows
- Monitor your system's available RAM 
- Stop before running out of memory!

**Hints:**
- Use `np.arange()` or `np.zeros()` to create large arrays efficiently
- Calculate target array size: 10 GB ÷ 8 bytes per int64 = ~1.25 billion elements
- Consider creating arrays in chunks if needed

### 🧮 **Advanced Challenge**
Compare the performance of these array creation methods for building large arrays:
1. Using `np.append()` in a loop (small arrays only!)
2. Pre-allocating with `np.zeros()` then filling values
3. Using `np.arange()` directly
4. Using `np.concatenate()` with smaller pre-built chunks

Time each method and explain the performance differences!